In [1]:
# getting OAuth access token
# acquire credential from https://platform.here.com/admin/apps/

import binascii  # To convert data into ASCII
import hashlib  # To generate SHA256 digest
import hmac  # To implement HMAC algorithm
import json
import time  # To generate the OAuth timestamp
import urllib.parse  # To URLencode the parameter string
from base64 import b64encode  # To encode binary data into Base64

import requests  # To make HTTP requests


def create_parameter_string(grant_type, oauth_consumer_key, oauth_nonce, oauth_signature_method, oauth_timestamp,
                            oauth_version):
    parameter_string = ''
    parameter_string = parameter_string + 'grant_type=' + grant_type
    parameter_string = parameter_string + '&oauth_consumer_key=' + oauth_consumer_key
    parameter_string = parameter_string + '&oauth_nonce=' + oauth_nonce
    parameter_string = parameter_string + '&oauth_signature_method=' + oauth_signature_method
    parameter_string = parameter_string + '&oauth_timestamp=' + oauth_timestamp
    parameter_string = parameter_string + '&oauth_version=' + oauth_version
    return parameter_string


def create_signature(secret_key, signature_base_string):
    encoded_string = signature_base_string.encode()
    encoded_key = secret_key.encode()
    temp = hmac.new(encoded_key, encoded_string, hashlib.sha256).hexdigest()
    byte_array = b64encode(binascii.unhexlify(temp))
    return byte_array.decode()


with open('credentials.properties', mode='r') as credential_properties:  # From credentials.properties file
    lines = credential_properties.readlines()
    here_user_id = lines[0].split(' = ')[1][:-1]
    here_client_id = lines[1].split(' = ')[1][:-1]
    here_access_key_id = lines[2].split(' = ')[1][:-1]
    here_access_key_secret = lines[3].split(' = ')[1][:-1]
    here_token_endpoint_url = lines[4].split(' = ')[1][:-1]

    grant_type = 'client_credentials'

    oauth_nonce = str(int(time.time() * 1000))
    oauth_signature_method = 'HMAC-SHA256'
    oauth_timestamp = str(int(time.time()))
    oauth_version = '1.0'

    parameter_string = create_parameter_string(grant_type, here_access_key_id, oauth_nonce, oauth_signature_method,
                                               oauth_timestamp, oauth_version)
    encoded_parameter_string = urllib.parse.quote(parameter_string, safe='')

    encoded_base_string = 'POST' + '&' + urllib.parse.quote(here_token_endpoint_url, safe='')
    encoded_base_string = encoded_base_string + '&' + encoded_parameter_string

    signing_key = here_access_key_secret + '&'

    oauth_signature = create_signature(signing_key, encoded_base_string)
    encoded_oauth_signature = urllib.parse.quote(oauth_signature, safe='')

    body = {'grant_type': '{}'.format(grant_type)}

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': 'OAuth oauth_consumer_key="{0}",oauth_nonce="{1}",oauth_signature="{2}",oauth_signature_method="HMAC-SHA256",oauth_timestamp="{3}",oauth_version="1.0"'.format(
            here_access_key_id, oauth_nonce, encoded_oauth_signature, oauth_timestamp)
    }

    response = requests.post(here_token_endpoint_url, data=body, headers=headers)
    print(response.text)

    oauth_result = json.loads(response.text)


{"access_token":"eyJhbGciOiJSUzUxMiIsImN0eSI6IkpXVCIsImlzcyI6IkhFUkUiLCJhaWQiOiJLdjBGN2l5NzNjZnoxOHRQdXYxOCIsImlhdCI6MTcxMDczMDg5NiwiZXhwIjoxNzEwODE3Mjk2LCJraWQiOiJqMSJ9.ZXlKaGJHY2lPaUprYVhJaUxDSmxibU1pT2lKQk1qVTJRMEpETFVoVE5URXlJbjAuLl9Oemo4T1ZTaXl3OUhBV0hKNW1mMFEuTFJubWF3cl9iUVEzRUl0dWZNRk9oQmxwdG9QdndWOWVDajhMek9mT0FlYjVaeWdZbUR5QW9waG0tQ2NHUHZQZ0JGaTFVbHhfUmxiMzZJZ1RxanJCdGZjWEMzT1dtR0xmSmdXcFJNbkRELWFYb0Z5aTR0a2tudjJPLUNleDUwMFo4OGZJNkFxVkFuVXMwcUVOclpRbVhRLndnRENxdUVVZ3BzNlhKa3BpdWZVVWJoS2JlajFsclJ3MHIzamNZVVQxZUk.ooMq89l18hD04wWEAUL2Pq9wnlw9AFrviJpLI0vnQz88muxbcyxcRpoY_PiL-BshdkydiE6NDHJehiJD4HmW4P8EpDlKlWXafHnNnc58qI90Kh3cnJ63L9C7hbT_MG8BoThfPYNe6FysGTcw3bdNj95L_SqJybEnGEaRjbM98uSChwlTNDhRMf-uW19-8zrWJi7J6HcF7DEmT-FxjGEPvpW6mcdshbZG9jzky9f732xtCLqukAzN1TqrwrMMic2VFBWsnW4_t8naycBC8RSIMunrjP2ALeng5xV_-pUnJkpIxGXRIp4XNDufzrGX_EGWD3dXf75J_q84YewHMU8tYg","token_type":"bearer","expires_in":86399}


In [2]:
# delivery target listing

import csv
destinations = 'destinations_v2.csv'
destination_list = []
f = open(destinations, encoding='utf-8')
csv = csv.DictReader(f)
for row in csv:
    destination_list.append(row)
print(destination_list)

[{'customer_id': 'c000001', 'name': '肯德基-花蓮中正餐廳', 'latitude': '23.97929', 'longitude': '121.61043', 'job_type': 'delivery', 'duration': '600', 'mass': '100', 'size': '1', 'priority': '2', 'time_start': '2024-11-25T01:00:00Z', 'time_end': '2024-11-25T09:00:00Z', 'task_fulfiller': '', 'estimated_time_of_arrival': '', 'actual_time_of_arrival': '', 'task_status': ''}, {'customer_id': 'c000002', 'name': '肯德基-台北台大餐廳', 'latitude': '25.01707', 'longitude': '121.53325', 'job_type': 'delivery', 'duration': '300', 'mass': '50', 'size': '1', 'priority': '2', 'time_start': '2024-11-25T01:00:00Z', 'time_end': '2024-11-25T09:00:00Z', 'task_fulfiller': '', 'estimated_time_of_arrival': '', 'actual_time_of_arrival': '', 'task_status': ''}, {'customer_id': 'c000003', 'name': '肯德基-彰化員林餐廳', 'latitude': '23.95943', 'longitude': '120.57111', 'job_type': 'delivery', 'duration': '600', 'mass': '100', 'size': '1', 'priority': '2', 'time_start': '2024-11-25T01:00:00Z', 'time_end': '2024-11-25T09:00:00Z', 'task_f

In [3]:
# delivery vehicle listing

import csv

destinations = 'fleet.csv'
f = open(destinations, encoding='utf-8')
csv = csv.reader(f)
for row in csv:
    print(row)

['vehicle_id', 'type', 'name', 'cost_distance', 'cost_time', 'cost_fixed', 'shift_start', 'shift_end', 'start_latitude', 'start_longitude', 'end_latitude', 'end_longitude', 'break_start', 'break_end', 'break_duration', 'capacity_mass', 'capacity_size', 'max_distance', 'shift_time', 'amount']
['isuzu', 'truck', 'ISUZU_NLR', '0.004', '0.033', '1600', '2024-11-25T00:00:00Z', '2024-11-25T09:00:00Z', '25.0633628527129', '121.5517443657892', '24.970504', '121.2516', '2024-11-25T04:00:00Z', '2024-11-25T05:00:00Z', '3600', '3500', '40', '600000', '28800', '5']
['mitsubishi', 'truck', 'MITSUBISHI_CANTER', '0.005', '0.04', '2000', '2024-11-25T00:00:00Z', '2024-11-25T09:00:00Z', '22.570109085287', '120.34168946760741', '24.970504', '121.2516', '2024-11-25T04:00:00Z', '2024-11-25T05:00:00Z', '3600', '3500', '50', '600000', '28800', '5']


In [4]:
# composing HERE Tour Planning API request body

import csv
import json

destinations = 'destinations_v2.csv'
vehicles = 'fleet.csv'

jobs = []
types = []
profiles = []

with open(destinations, encoding='utf-8') as destinations:
    reader = csv.DictReader(destinations)
    for row in reader:
        job_type = row['job_type']
        if job_type == 'delivery':
            jobs.append({
                "id": row['customer_id'],
                "tasks": {
                "deliveries": [
                    {
                    "places": [
                        {
                        "times": [
                            [row['time_start'],
                            row['time_end']]
                        ],
                        "location": {
                            "lat": float(row['latitude']),
                            "lng": float(row['longitude'])
                        },
                        "duration": int(row['duration'])
                        }
                    ],
                    "demand": [int(row['size'])]
                    }   
                ],
                },
                "priority": int(row['priority'])
            }
        )
        


with open(vehicles, encoding='utf-8') as vehicle_list:
    reader = csv.DictReader(vehicle_list)
    for row in reader:
        types.append({
            "id": row['vehicle_id'],
            "profile": row['name'],
            "costs": {
                "fixed": float(row['cost_fixed']),
                "distance": float(row['cost_distance']),
                "time": float(row['cost_time'])
            },
            "shifts": [
                {
                    "start": {
                        "time": row['shift_start'],
                        "location": {
                            "lat": float(row['start_latitude']),
                            "lng": float(row['start_longitude'])
                        }
                    },
                    "end": {
                        "time": row['shift_end'],
                        "location": {
                            "lat": float(row['end_latitude']),
                            "lng": float(row['end_longitude'])
                        }
                    },
                    "breaks": [
                        {
                            "times": [
                                [
                                    row['break_start'],
                                    row['break_end']
                                ]
                            ],
                            "duration": int(row['break_duration']),
                        }
                    ]
                }
            ],
            "capacity": [
                int(row['capacity_mass']),
                int(row['capacity_size'])
            ],
            "skills": [
                "fridge"
            ],
            "limits": {
                "maxDistance": int(row['max_distance']),
                "shiftTime": int(row['shift_time'])
            },
            "amount": int(row['amount'])
        })
        profiles.append({
            "name": row['name'],
            "type": row['type']
        })

problem = {
    "plan": {
        "jobs": jobs,
    },
    "fleet": {
        "types": types,
        "profiles": profiles
    },
    "configuration": {
        "termination": {
            "maxTime": 240,
            "stagnationTime": 1
         }
    }
}

data = json.dumps(problem, ensure_ascii=False)

print(data)

{"plan": {"jobs": [{"id": "c000001", "tasks": {"deliveries": [{"places": [{"times": [["2024-11-25T01:00:00Z", "2024-11-25T09:00:00Z"]], "location": {"lat": 23.97929, "lng": 121.61043}, "duration": 600}], "demand": [1]}]}, "priority": 2}, {"id": "c000002", "tasks": {"deliveries": [{"places": [{"times": [["2024-11-25T01:00:00Z", "2024-11-25T09:00:00Z"]], "location": {"lat": 25.01707, "lng": 121.53325}, "duration": 300}], "demand": [1]}]}, "priority": 2}, {"id": "c000003", "tasks": {"deliveries": [{"places": [{"times": [["2024-11-25T01:00:00Z", "2024-11-25T09:00:00Z"]], "location": {"lat": 23.95943, "lng": 120.57111}, "duration": 600}], "demand": [1]}]}, "priority": 2}, {"id": "c000004", "tasks": {"deliveries": [{"places": [{"times": [["2024-11-25T01:00:00Z", "2024-11-25T09:00:00Z"]], "location": {"lat": 24.98914, "lng": 121.50996}, "duration": 300}], "demand": [1]}]}, "priority": 2}, {"id": "c000005", "tasks": {"deliveries": [{"places": [{"times": [["2024-11-25T01:00:00Z", "2024-11-25T09

In [5]:
# request HERE Tour Planning API V3 to get planning results

import requests

tour_planning_api_url = "https://tourplanning.hereapi.com/v3/problems"

headers = {
  'Content-Type': 'application/json',
  'Authorization': '{} {}'.format(oauth_result['token_type'], oauth_result['access_token'])
}

r = requests.request('POST', tour_planning_api_url, headers=headers, data = data)
j = json.loads(r.text)

print('result:\n{}'.format(r.text))
print('response_time: {}'.format(r.elapsed.total_seconds()))

result:
{"statistic":{"cost":41069.052,"distance":2725485,"duration":284221,"times":{"driving":193021,"serving":66000,"waiting":0,"stopping":0,"break":25200}},"tours":[{"vehicleId":"isuzu_5","typeId":"isuzu","stops":[{"time":{"arrival":"2024-11-25T00:00:00Z","departure":"2024-11-25T00:49:08Z"},"load":[26],"activities":[{"jobId":"departure","type":"departure","location":{"lat":25.0633628527129,"lng":121.5517443657892},"time":{"start":"2024-11-25T00:00:00Z","end":"2024-11-25T00:49:08Z"}}],"location":{"lat":25.0633628527129,"lng":121.5517443657892},"distance":0},{"time":{"arrival":"2024-11-25T01:02:32Z","departure":"2024-11-25T01:17:32Z"},"load":[24],"activities":[{"jobId":"c000118","type":"delivery","location":{"lat":25.04745,"lng":121.51614},"time":{"start":"2024-11-25T01:02:32Z","end":"2024-11-25T01:07:32Z"}},{"jobId":"c000117","type":"delivery","location":{"lat":25.04745,"lng":121.51614},"time":{"start":"2024-11-25T01:07:32Z","end":"2024-11-25T01:17:32Z"}}],"location":{"lat":25.04745,

In [6]:
import json
import random

import flexpolyline
import folium
import requests
from folium.plugins import BeautifyIcon, AntPath

import solution_v3 as solution


def get_route(ori_lat, ori_lon, dest_lat, dest_lon, route_mode, departure_time):
    route_url = 'https://router.hereapi.com/v8/routes?'
    wp0 = ('{},{}'.format(ori_lat, ori_lon))
    wp1 = ('{},{}'.format(dest_lat, dest_lon))
    route_options = '&transportMode={}&departureTime={}&return=polyline'.format(route_mode, departure_time)
    url = route_url + 'apiKey=' + apikey + '&origin=' + wp0 + '&destination=' + wp1 + route_options
    json_result = json.loads(requests.get(url).text)
    routes = json_result['routes']
    route_shape = []
    decoded_polyline = flexpolyline.decode(routes[0]['sections'][0]['polyline'])
    return decoded_polyline


with open('apikey.txt', mode='r') as apikey_txt:
    apikey = apikey_txt.read()

m = folium.Map(
    tiles='https://1.base.maps.ls.hereapi.com/maptile/2.1/maptile/newest/normal.day/{z}/{x}/{y}/256/png8?apiKey=' + apikey,
    location=[23, 121],
    max_zoom=20,
    attr='(c)2021 HERE'
)

folium.Marker([25.0633628527129, 121.5517443657892],
              icon=BeautifyIcon(icon='home', iconShape='marker', background_color='#000000', text_color='#FFFFFF'),
              popup='Home').add_to(m)
folium.Marker([22.570109085287, 120.34168946760741],
              icon=BeautifyIcon(icon='home', iconShape='marker', background_color='#000000', text_color='#FFFFFF'),
              popup='Home').add_to(m)

bounds = []

customer_dict = {}
unreachable_customer_names = []
solution = solution.solution_from_dict(j)
unassigned = solution.unassigned
unassigned_feature_group = folium.map.FeatureGroup(name='Unassigned', overlay=True, control=True, show=True)
icon_color = '#FF0000'
for destination_dict in destination_list:
    customer_dict[destination_dict['customer_id']] = destination_dict['name']
    if len(unassigned) > 0:
        for unassigned_destination in unassigned:
            job_id = unassigned_destination.job_id
            customer_name = ''
            if destination_dict['customer_id'] == job_id:
                customer_name = destination_dict['name']
                unreachable_customer_names.append(customer_name)
                folium.Marker([destination_dict['latitude'], destination_dict['longitude']],
                              icon=BeautifyIcon(icon='ban', iconShape='marker', background_color=icon_color,
                                                border_width=2), popup='{}/{}<br>{}'.format(job_id, customer_name,
                                                                                            unassigned_destination.reasons.__getitem__(
                                                                                                0).description)).add_to(
                    unassigned_feature_group)
unassigned_feature_group.add_to(m)
print("unreachable customers: ")
print(unreachable_customer_names)

tour_index = 0
tour_list = []
vehicle_list = []
while tour_index < len(solution.tours):
    icon_color = '#'
    i = 0
    while i < 6:
        icon_color += hex(random.randint(6, 16))[-1]
        i += 1
    tour = solution.tours.__getitem__(tour_index)
    vehicle_id = tour.vehicle_id
    vehicle_list.append(vehicle_id)
    statistic = tour.statistic
    print("\ncalculating routes for tour: {} / vehicle: {} / stops: {} / cost: {} / distance: {} / duration: {}".format(
        tour_index, vehicle_id, len(tour.stops), int(statistic.cost), statistic.distance, statistic.duration))
    feature_group = folium.map.FeatureGroup(name=vehicle_id, overlay=True, control=True, show=True)
    type_id = tour.type_id
    stops = tour.stops
    stop_index = 0
    stop_list = []
    trip_start_timestamp = 0
    trip_end_timestamp = 0
    movement_time = 0
    while stop_index < len(stops):
        stop = stops.__getitem__(stop_index)
        previous_stop = stops.__getitem__(stop_index - 1)
        stop_location = stop.location.to_dict()
        stop_time = stop.time
        stop_time_arrival = stop.time.arrival
        stop_time_departure = stop.time.departure
        stop_time_arrival_from_trip_start = 0
        stop_time_departure_from_trip_start = 0
        stop_activities = stop.activities

        if stop_index == 0:
            trip_start_timestamp = stop_time_departure
        else:
            stop_time_arrival_from_trip_start = stop_time_arrival - trip_start_timestamp
            stop_time_departure_from_trip_start = stop_time_departure - trip_start_timestamp
        print('\tstop_time_arrival_from_trip_start: {}'.format(stop_time_arrival_from_trip_start))
        print('\tstop_time_departure_from_trip_start: {}'.format(stop_time_departure_from_trip_start))
        if stop_index > 0:
            previous_stop = stops.__getitem__(stop_index - 1)
            movement_time = stop_time.arrival - previous_stop.time.departure
            movement_time.seconds
            print('\t~ movement time: {} ~'.format(movement_time))
        for stop_activity in stop_activities:
            job_id = stop_activity.job_id
            for destination_dict in destination_list:
                if destination_dict['customer_id'] == job_id:
                    stop_list.append({'stop_index': stop_index, 'job_id': job_id, 'destination': destination_dict})
                elif job_id == 'departure' or job_id == 'arrival' or job_id == 'break':
                    stop_list.append({'stop_index': stop_index, 'job_id': job_id,
                                      'destination': {'latitude': stop_location['lat'],
                                                      'longitude': stop_location['lng']}})
                    break
            customer_name = customer_dict.get(job_id)
            stop_load = stop.load
            if stop_index < len(stops) - 1:
                route_shape = get_route(stop_location['lat'], stop_location['lng'],
                                        stops.__getitem__(stop_index + 1).location.to_dict()['lat'],
                                        stops.__getitem__(stop_index + 1).location.to_dict()['lng'], 'truck',
                                        stop_time_departure.strftime('%Y-%m-%dT%H:%M:%SZ'))
                if stop_index > 0:
                    folium.Marker([stop_location['lat'], stop_location['lng']],
                                  icon=BeautifyIcon(icon='flag', iconShape='marker', background_color=icon_color,
                                                    border_width=2),
                                  popup='Vehicle ID: {}<br>Job ID: {}/{}<br>Arrival：{}<br>Departure：{}'.format(
                                      vehicle_id, job_id, customer_name, stop_time_arrival,
                                      stop_time_departure)).add_to(feature_group)
                shape_point_index = 0
                bounds.append([stop_location['lat'], stop_location['lng']])
                shape_point_list = []
                while shape_point_index < len(route_shape):
                    shape_point = route_shape[shape_point_index]
                    shape_point_list.append(shape_point)
                    shape_point_index += 1
                AntPath(shape_point_list, color=icon_color, weight=4, opacity=1).add_to(feature_group)
            print('{} --> {} / {} / {} / arr: {} dep: {} '.format(stop_index, job_id, customer_name,
                                                                  [stop_location['lat'], stop_location['lng']],
                                                                  stop_time.arrival, stop_time.departure))

        stop_index += 1
    statistic = tour.statistic
    tour_list.append({'vehicle_id': vehicle_id, 'stop_list': stop_list})
    feature_group.add_to(m)
    tour_index += 1


unreachable customers: 
['肯德基-花蓮中正餐廳', '肯德基-宜蘭羅東餐廳', '肯德基-台東新生餐廳', '肯德基-台南中華餐廳', '肯德基-屏東逢甲餐廳', '肯德基-南投埔里餐廳', '肯德基-台南中華西餐廳', '肯德基-斗六民生餐廳', '肯德基-屏東自由餐廳', '肯德基-彰化鹿港餐廳', '肯德基-汐止中興餐廳', '肯德基-羅東興東餐廳']

calculating routes for tour: 0 / vehicle: isuzu_5 / stops: 27 / cost: 2967 / distance: 106113 / duration: 28584
	stop_time_arrival_from_trip_start: 0
	stop_time_departure_from_trip_start: 0
0 --> departure / None / [25.0633628527129, 121.5517443657892] / arr: 2024-11-25 00:00:00+00:00 dep: 2024-11-25 00:49:08+00:00 
	stop_time_arrival_from_trip_start: 0:13:24
	stop_time_departure_from_trip_start: 0:28:24
	~ movement time: 0:13:24 ~
1 --> c000118 / 肯德基-台北車站B2餐廳 / [25.04745, 121.51614] / arr: 2024-11-25 01:02:32+00:00 dep: 2024-11-25 01:17:32+00:00 
1 --> c000117 / 肯德基-台北車站B1餐廳 / [25.04745, 121.51614] / arr: 2024-11-25 01:02:32+00:00 dep: 2024-11-25 01:17:32+00:00 
	stop_time_arrival_from_trip_start: 0:33:19
	stop_time_departure_from_trip_start: 0:38:19
	~ movement time: 0:04:55 ~
2 --> c000078 /

In [7]:
folium.LayerControl(collapsed=True, hideSingleBase=True).add_to(m)
m.fit_bounds(bounds)
m.save('here_tour_planning_v3_result_map.html')
m